In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

# Data Preparation

In [ ]:
X_train = pd.read_csv("X_train.txt", header=None,  delim_whitespace=True)
y_train = pd.read_csv("y_train.txt", header=None, delim_whitespace=True )
X_test = pd.read_csv("X_test.txt", header=None, delim_whitespace=True )
y_test = pd.read_csv("y_test.txt", header=None, delim_whitespace=True )
subject_test = pd.read_csv("subject_test.txt", header=None, delim_whitespace=True )
subject_train = pd.read_csv("subject_train.txt", header=None, delim_whitespace=True )
features = pd.read_csv("features.txt", header=None, delim_whitespace=True )

In [ ]:
body_acc_x_train = pd.read_csv("body_acc_x_train.txt", header=None,  delim_whitespace=True)
body_acc_y_train = pd.read_csv("body_acc_y_train.txt", header=None,  delim_whitespace=True)
body_acc_z_train = pd.read_csv("body_acc_z_train.txt", header=None,  delim_whitespace=True)
body_gyro_x_train = pd.read_csv("body_gyro_x_train.txt", header=None,  delim_whitespace=True)
body_gyro_y_train = pd.read_csv("body_gyro_y_train.txt", header=None,  delim_whitespace=True)
body_gyro_z_train = pd.read_csv("body_gyro_z_train.txt", header=None,  delim_whitespace=True)
total_acc_x_train = pd.read_csv("body_gyro_x_train.txt", header=None,  delim_whitespace=True)
total_acc_y_train = pd.read_csv("body_gyro_y_train.txt", header=None,  delim_whitespace=True)
total_acc_z_train = pd.read_csv("body_gyro_z_train.txt", header=None,  delim_whitespace=True)

body_acc_x_test  = pd.read_csv("body_acc_x_test.txt", header=None,  delim_whitespace=True)
body_acc_y_test  = pd.read_csv("body_acc_y_test.txt", header=None,  delim_whitespace=True)
body_acc_z_test  = pd.read_csv("body_acc_z_test.txt", header=None,  delim_whitespace=True)
body_gyro_x_test = pd.read_csv("body_gyro_x_test.txt", header=None,  delim_whitespace=True)
body_gyro_y_test = pd.read_csv("body_gyro_y_test.txt", header=None,  delim_whitespace=True)
body_gyro_z_test = pd.read_csv("body_gyro_z_test.txt", header=None,  delim_whitespace=True)
total_acc_x_test = pd.read_csv("body_gyro_x_test.txt", header=None,  delim_whitespace=True)
total_acc_y_test = pd.read_csv("body_gyro_y_test.txt", header=None,  delim_whitespace=True)
total_acc_z_test = pd.read_csv("body_gyro_z_test.txt", header=None,  delim_whitespace=True)

In [ ]:
feature = features
feature.drop(0,inplace=True, axis=1)

In [ ]:
#inseriamo l'intestazione al dataset
lista=[]
feat_transpa = feature.transpose()
for i in range(561):
    lista.append(feat_transpa.iloc[0][i])
X_test.columns=lista    
X_train.columns=lista

In [ ]:
# eliminiamo le colonne che contengono la stima mad in quanto quasi uguale a dev.std

stringa="mad()"
for col in X_train.columns:
    if(stringa in col):
        X_train.drop(labels=col, axis=1, inplace=True)
for col in X_test.columns:
    if(stringa in col):
        X_test.drop(labels=col, axis=1, inplace=True)
#for col in features:
#    if (stringa in col):
#        features.drop(labels=col,axis=1,inplace=True)

## Sax_transformation

In [ ]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.piecewise import SymbolicAggregateApproximation

ts=body_gyro_z_train

In [ ]:
#normalizz
scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)  # Rescale time series # Normalizza una riga
ts_norm = scaler.fit_transform(ts)
ts_norm.shape

In [ ]:
n_paa_segments = 24
n_sax_symbols = 6
sax = SymbolicAggregateApproximation(n_segments=n_paa_segments, alphabet_size_avg=n_sax_symbols)
ts_sax = sax.fit_transform(ts_norm)
ts_sax_inv = sax.inverse_transform(ts_sax)

In [ ]:
np.unique(ts_sax_inv)    #Per vedere i valori soglia

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(ts_norm[2528], label='Original') #Max var Walking-upstair 2528
plt.plot(ts_sax_inv[2528], label='SAX_Approx') #min var Sitting 3288
#plt.plot(ts_sax[0], label='SAX')
plt.legend(loc='best')
plt.show()

## Sequential Pattern Mining

In [ ]:
#!pip install prefixspan

In [ ]:
from prefixspan import PrefixSpan

In [ ]:
ts_sax=ts_sax.reshape(7352,24)
ps = PrefixSpan(ts_sax)

In [ ]:
ts_sax

In [ ]:
b=ps.topk(7352)

In [ ]:
# [( 2, [0]) -> vuol dire che '0' compare 2 volte
# [( 2, [1, 1]) -> vuol dire che '11' compare 2 volte
a= ps.frequent(735)  # param is min freq absolute--> 735 corrisponde al minsupport del 10%; 1470 = minsup del 20%

In [ ]:
#In questo caso se vogliamo utilizzare "b" che rappresenta la topk
count = 0
newarr = []
for i in range(len(b)):
    if len(b[i][1]) == 5 and count < 6:
        newarr.append(b[i])
        count += 1

In [ ]:
#IN questo caso se vogliamo utilizzare "a" che rappresenta i pattern con minsup specificato
#dove prima dobbiamo ordinare i valori per support, in modo tale che i primi che becca per quella dimensione siamo sicuri
#che saranno quelli con support maggiore

result = sorted(a, reverse=True)
result = tuple(result)
#print('Sorted Tuple :', result)

count = 0
newarr = []
for i in range(len(result)):
    if len(result[i][1]) == 5 and count < 5:
        newarr.append(result[i])
        count += 1

In [ ]:
result #stampa tutti i risultati

In [ ]:
newarr #stampa i risultati specificando lunghezza pattern e freq

In [ ]:
#settiamo il new arr come una stringa che possa essere poi scansionata all interno del dataset 
import re

arrnew=[]
for i in range(len(newarr)):
    print(newarr[i][1])
    string = str(newarr[i][1])
    string = re.sub("\[|\]", "", string)
    string = re.sub("\,", "", string)
    arrnew.append(string)
arrnew[0]    

In [ ]:
#modifichiamo anche ts_sax
ts_sax_nuovo=[]
for i in range(len(ts_sax)):
    string = str(ts_sax[i])
    string = re.sub("\[|\]", "", string)
    ts_sax_nuovo.append(string)

In [ ]:
pattern_found = []
index1 = []
index2 = []
index3 = []
index4 = []
index5 = []
index6 = []

count = 0
for i in range(len(arrnew)):
    count = 0
    for j in range(len(ts_sax_nuovo)):
        #print(ts_sax_nuovo[j], arrnew[i]
        if arrnew[i] in str(ts_sax_nuovo[j]):
            count +=1
            if i == 0:
                index1.append(j)
            if i == 1:
                index2.append(j)
            if i == 2:
                index3.append(j)
            if i == 3:
                index4.append(j)
            if i == 4:
                index5.append(j)
            #if i == 5:
                #index6.append(j)
    pattern_found.append(count)

In [ ]:
pattern_found

## Dataset con ricorrenze dei pattern trovati

In [ ]:
y_train_sbilanciato=y_train
y_train[0][40]

In [ ]:
#pattern 1
a1 = np.count_nonzero(y_train_sbilanciato[0][index1]==1)
a2 = np.count_nonzero(y_train_sbilanciato[0][index1]==2)
a3 = np.count_nonzero(y_train_sbilanciato[0][index1]==3)
a4 = np.count_nonzero(y_train_sbilanciato[0][index1]==4)
a5 = np.count_nonzero(y_train_sbilanciato[0][index1]==5)
a6 = np.count_nonzero(y_train_sbilanciato[0][index1]==6)

#pattern 2
b1 = np.count_nonzero(y_train_sbilanciato[0][index2]==1)
b2 = np.count_nonzero(y_train_sbilanciato[0][index2]==2)
b3 = np.count_nonzero(y_train_sbilanciato[0][index2]==3)
b4 = np.count_nonzero(y_train_sbilanciato[0][index2]==4)
b5 = np.count_nonzero(y_train_sbilanciato[0][index2]==5)
b6 = np.count_nonzero(y_train_sbilanciato[0][index2]==6)

#pattern 3
c1 = np.count_nonzero(y_train_sbilanciato[0][index3]==1)
c2 = np.count_nonzero(y_train_sbilanciato[0][index3]==2)
c3 = np.count_nonzero(y_train_sbilanciato[0][index3]==3)
c4 = np.count_nonzero(y_train_sbilanciato[0][index3]==4)
c5 = np.count_nonzero(y_train_sbilanciato[0][index3]==5)
c6 = np.count_nonzero(y_train_sbilanciato[0][index3]==6)


#pattern 4
d1 = np.count_nonzero(y_train_sbilanciato[0][index4]==1)
d2 = np.count_nonzero(y_train_sbilanciato[0][index4]==2)
d3 = np.count_nonzero(y_train_sbilanciato[0][index4]==3)
d4 = np.count_nonzero(y_train_sbilanciato[0][index4]==4)
d5 = np.count_nonzero(y_train_sbilanciato[0][index4]==5)
d6 = np.count_nonzero(y_train_sbilanciato[0][index4]==6)


#pattern 5
e1 = np.count_nonzero(y_train_sbilanciato[0][index5]==1)
e2 = np.count_nonzero(y_train_sbilanciato[0][index5]==2)
e3 = np.count_nonzero(y_train_sbilanciato[0][index5]==3)
e4 = np.count_nonzero(y_train_sbilanciato[0][index5]==4)
e5 = np.count_nonzero(y_train_sbilanciato[0][index5]==5)
e6 = np.count_nonzero(y_train_sbilanciato[0][index5]==6)


##pattern 6
#f1 = np.count_nonzero(y_train_sbilanciato[0][index6]==1)
#f2 = np.count_nonzero(y_train_sbilanciato[0][index6]==2)
#f3 = np.count_nonzero(y_train_sbilanciato[0][index6]==3)
#f4 = np.count_nonzero(y_train_sbilanciato[0][index6]==4)
#f5 = np.count_nonzero(y_train_sbilanciato[0][index6]==5)
#f6 = np.count_nonzero(y_train_sbilanciato[0][index6]==6)
#

In [ ]:
d= {"P1" : [a1,a2,a3,a4,a5,a6], "P2" : [b1,b2,b3,b4,b5,b6], "P3" : [c1,c2,c3,c4,c5,c6], "P4" : [d1,d2,d3,d4,d5,d6], "P5" : [e1,e2,e3,e4,e5,e6]}
new=pd.DataFrame(data=d)

In [ ]:
new